In [6]:
import gym
env_name = 'LunarLander-v2'
env = gym.make(env_name)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [2]:
import tensorflow as tf
import tflearn
import numpy as np
tf.reset_default_graph() # THIS IS NECESSARY BEFORE MAKING NEW SESSION TO STOP IT ERRORING!!
try:
    sess
except:
    pass
else:
    sess.close()
    del sess
sess = tf.InteractiveSession()

n_inputs = env.observation_space.shape[0]
n_hidden = 10  # it's a simple task, we don't need more than this
n_hlayers = 3
n_outputs = env.action_space.n # only outputs the probability of accelerating left
initializer = tf.contrib.layers.variance_scaling_initializer()
LR = 1e-2

# 2. Build the neural network
X = tf.placeholder(tf.float32, shape=[None, n_inputs])
Y = tf.placeholder(tf.int32, shape=[None])

layer = X
for _ in range(n_hlayers):
    layer = tf.layers.dense(layer, n_hidden, activation=tf.nn.relu,
                         kernel_initializer=initializer)

logits = tf.layers.dense(layer, n_outputs,
                          kernel_initializer=initializer)
outputs = tf.nn.softmax(logits)
# 3. Select a random action based on the estimated probabilities
# p_left_and_right = tf.concat(axis=1, values=[outputs, 1 - outputs])
action = tf.multinomial(tf.log(outputs), num_samples=1)[0]

loss = tf.losses.sparse_softmax_cross_entropy(labels=Y, logits=logits)
train_step = tf.train.AdamOptimizer(LR).minimize(loss)
    
tf.global_variables_initializer().run()
saver = tf.train.Saver()

# a = np.expand_dims(np.random.rand(4),axis=0)
# print(sess.run([logits,outputs,action], feed_dict={X: a}))


//anaconda/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
observation = env.reset()
act = sess.run(action, feed_dict={X: np.expand_dims(observation,axis=0)})[0]
observation, reward, done, info = env.step(act)
print(act)
print(observation)
print(reward)

1
[-0.00451479  0.9395422  -0.23530638 -0.04853535  0.00741422  0.09692287
  0.          0.        ]
-1.9571213077518468


In [5]:
render = False

i = 0
rewards_t = []
    
while i < 10000:
    observation = env.reset()
    acts = []
    obvs = []
    reward_t = 0
    for t in range(1000):
        if render:
            env.render()
            print(reward)
        act = sess.run(action, feed_dict={X: np.expand_dims(observation,axis=0)})[0]
        acts.append(act)
        obvs.append(observation)
        observation, reward, done, info = env.step(act)
        reward_t += reward
        if done:
            rewards_t.append(reward_t)
            if (reward_t > (np.mean(rewards_t[-10:-1]) + 0.5 * np.std(rewards_t[-10:-1]))):
                print('training! %d' % reward_t)
                sess.run(train_step, feed_dict={X: obvs, Y:acts})
                max_t = t
                i += 1
            break
#     if np.mean(rewards_t[-10:-1]) > 180:
#        break
            
# saver.save(sess, "./my_policy_net_basic.ckpt")

env.close()

//anaconda/envs/tensorflow/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
//anaconda/envs/tensorflow/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
//anaconda/envs/tensorflow/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
//anaconda/envs/tensorflow/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
//anaconda/envs/tensorflow/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


training! -229
training! -154
training! -116
training! 4
training! -137
training! -157
training! -165
training! -132
training! -145
training! -170
training! -109
training! -153
training! -147
training! -124
training! -163
training! -61
training! -69
training! -134
training! -6
training! -112
training! -116
training! -150
training! -161
training! -161
training! -163
training! -136
training! -158
training! -155
training! -146
training! -123
training! -140
training! -151
training! -149
training! -133
training! -146
training! -132
training! -62
training! -143
training! -154
training! -174
training! -181
training! -158
training! -107
training! -135
training! -149
training! -159
training! -163
training! -176
training! -152
training! -122
training! -154
training! -114
training! -176
training! -144
training! -142
training! -114
training! -126
training! -114
training! -133
training! -131
training! -139
training! -75
training! -106
training! -111
training! -147
training! -166
training! -46
train

training! -47
training! -62
training! -45
training! -30
training! -39
training! -45
training! -44
training! -14
training! -49
training! -62
training! -40
training! -41
training! -33
training! -59
training! -38
training! -39
training! -49
training! -44
training! -54
training! -24
training! -27
training! 10
training! -16
training! -14
training! -19
training! -27
training! -25
training! -22
training! -23
training! -18
training! -1
training! 4
training! -14
training! 7
training! 166
training! 80
training! 143
training! 215
training! 159
training! 143
training! 141
training! 217
training! -44
training! 132
training! 229
training! 9
training! -12
training! 7
training! -11
training! 21
training! -22
training! -26
training! 8


KeyboardInterrupt: 

In [9]:
rewardsum = 0
env = gym.make(env_name)
obs = env.reset()
for step in range(1000):
    env.render()
    action_val = action.eval(feed_dict={X: obs.reshape(1, n_inputs)})
    obs, reward, done, info = env.step(action_val[0])
    rewardsum += reward
    if done:
        break
env.close()
print(rewardsum)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
-106.87053276662306
